In [1]:
from datetime import datetime

import pandas as pd
import pyclip

from read_telega_dump import telega_dump_parse_essential

102050it [00:01, 100135.53it/s]


In [28]:
dump_path = r"/Users/dklmn/Documents/data/telega/result.json"
msgs_list = list(telega_dump_parse_essential(dump_path=dump_path))
# note: for data, taken from "export chat history", we do not have number of reactions 
msgs_list[0]


102050it [00:03, 31024.34it/s]


TelegaMessage(msg_id=80855, msg_date=datetime.datetime(2023, 1, 1, 10, 49, 43), user_name='Elmanov Anton', user_id='user881257459', reply_to_msg_id=None, msg_text='Если кто-то планировал на канатку в Сарысу.')

In [29]:
import json
# Custom serialization function for datetime objects
def date_to_json_serialize(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # or obj.strftime('%Y-%m-%d %H:%M:%S')
    raise TypeError("Type not serializable")

subset = msgs_list[-100:] # last n messages
dss = [msg.model_dump() for msg in subset]
json_string = json.dumps(dss, default=date_to_json_serialize, ensure_ascii=False, indent=4)
print(f'len str ={len(json_string)}')
pyclip.copy(json_string) # copy to clipboard for feeding to LLM

len str =30356


In [30]:

dd = (msg.model_dump() for msg in msgs_list)
df = pd.DataFrame.from_dict(dd)

In [31]:
df.shape

(102050, 6)

In [10]:
df.dtypes

msg_id                      int64
msg_date           datetime64[ns]
user_name                  object
user_id                    object
reply_to_msg_id           float64
msg_text                   object
dtype: object

### Overall statistics

In [11]:
df['dummi'] = 1 # there should be less ugly way to get different aggegations without grouping
agg_df = df.groupby('dummi').aggregate({'msg_id':'count','msg_date':['min','max'], 'user_id': 'nunique'}).reset_index()
msgs_cnt = agg_df[( 'msg_id', 'count')][0]
min_date = agg_df[('msg_date', 'min')][0]
max_date = agg_df[('msg_date', 'max')][0]
number_of_days = (max_date-min_date).days
unique_users =  agg_df[('user_id', 'nunique')][0]
del df['dummi'] 
cols = ["Metric name", "Value"]
dt_format = "%Y-%m-%d"
d_report = [
    {cols[0]: "Number or messages", cols[1]: msgs_cnt},
    {cols[0]: "Date interval", cols[1]: f'from {min_date.strftime(dt_format)} to {max_date.strftime(dt_format)}' },
    {cols[0]: "Users count", cols[1]: unique_users },
    {cols[0]: "Avg messages per day", cols[1]: round(msgs_cnt/number_of_days , 1) },
    ]
pd.DataFrame.from_dict(d_report)






,Metric name,Value
0,Number or messages,102050
1,Date interval,from 2023-01-01 to 2024-08-25
2,Users count,791
3,Avg messages per day,169.5


### assign date range if you want to restict dataset to some particular period for futher analysis, or just ignore the cell below 

In [31]:

# tz = timezone('Europe/Istanbul')
dt_range = (pd.Timestamp(year=2024, month=8, day=23), pd.Timestamp(datetime.now()))   # , tz=tm_zn        
number_of_days = (dt_range[1]-dt_range[0]).days
print(f'{number_of_days=}')
df = df[df.msg_date.between(*dt_range)]
df.shape


number_of_days=8


(307, 7)

In [32]:
df

,msg_id,msg_date,user_name,user_id,reply_to_msg_id,msg_text
0,80855,2023-01-01 10:49:43,Elmanov Anton,user881257459,NaN,Если кто-то планировал на канатку в Сарысу.
1,80856,2023-01-01 12:07:33,Vacum,user402560084,NaN,
2,80857,2023-01-01 12:12:36,Товарищ Белка,user431001350,NaN,
3,80858,2023-01-01 13:26:42,7amerlan,user494890771,NaN,кто уже был в lend of legends? когда лучше при...
4,80859,2023-01-01 13:56:52,Maria Kuznetsova,user872168334,NaN,кто-нибудь посмотрел уже аватара?\nя так понял...
...,...,...,...,...,...,...
102045,188889,2024-08-25 15:23:53,I am,user751268618,188888.0,Это школа
102046,188890,2024-08-25 15:34:34,ShamilNerd,user495977250,NaN,Да? Как так. С виду просто обычный дом. Мне ка...
102047,188891,2024-08-25 15:39:52,Rasul,user555411,188888.0,Я такую аэрографию хотел себе на автомобиль.
102048,188892,2024-08-25 15:45:27,Ekaterina Elm,user52026345,NaN,Там курсы же вроде


In [11]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()


ObjectApiResponse({'name': 'b5426ee70422', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'PFgTgTLvSU63_psQqF3g2w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


Step 4: Create Mappings and Index

    Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

    Each document is a collection of fields, which each have their own data type.

    We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored



In [105]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "msg_id": {"type": "integer"},
            "reply_to_msg_id": {"type": "integer"},
            "msg_text": {"type": "text", "analyzer": "russian"},
             "msg_date": {
                "type": "date"
                },
           # "msg_text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}


In [106]:
index_name = "telegram-messages"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'telegram-messages'})

In [108]:
subset = [x for x in raw_data if x['msg_date']> datetime(2024,8, 1)]

In [109]:
from  tqdm import tqdm
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
for doc in tqdm(subset):
    msg_text = doc["msg_text"]
    # Transforming the title into an embedding using the model
    # doc["msg_text_vector"] = model_st.encode(msg_text).tolist()
    doc['len'] = len(msg_text)
    doc["num_tokens"] = len(encoding.encode(msg_text))
    es_client.index(index=index_name, document=doc)


100%|██████████| 5246/5246 [00:26<00:00, 196.42it/s]


In [124]:
es_client.count(index=index_name)

ObjectApiResponse({'count': 5246, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

Test end users query

In [101]:
search_term = "криптообменники"
knn_query = {
    "field": "msg_text_vector",
    "query_vector": model_st.encode(search_term),
    "k": 5,
    "num_candidates": 10000, 
}

In [145]:
search_term = "сплав по реке"
text_query = {
    "query": {
        "match": {
            "msg_text": search_term
        }
    },
    "size": 20  # Adjust this number to include more documents
}

In [146]:
# res = es_client.search(index=index_name, knn=knn_query, source=["msg_text", "msg_date"])
res = es_client.search(index=index_name, body=text_query, source=["msg_text", "msg_date"])
res["hits"] #["hits"]

/var/folders/_s/0tpy0fpj29nd_8qm0ldhz6nw0000gn/T/ipykernel_52819/4052011558.py:2: DeprecationWarning: Received 'source' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es_client.search(index=index_name, body=text_query, source=["msg_text", "msg_date"])


{'total': {'value': 3, 'relation': 'eq'},
 'max_score': 7.9194794,
 'hits': [{'_index': 'telegram-messages',
   '_id': 'J_1XmpEBhq3eaWQ52YBG',
   '_score': 7.9194794,
   '_source': {'msg_date': '2024-08-09T12:56:43',
    'msg_text': 'Проблемы есть, но в сравнении с РФ незначительные, потому что их можно тут сплавить бигбиту или чему-то такому'}},
  {'_index': 'telegram-messages',
   '_id': '2v1XmpEBhq3eaWQ5yHy9',
   '_score': 5.9018955,
   '_source': {'msg_date': '2024-08-06T13:19:16',
    'msg_text': 'А кто-то ходил на Тунектепе недавно, ато вижу рекламу в канале, что был поход. Почему-то думал, что там закрыто после авариию Через русло реки значит тоже можно снова зайти?'}},
  {'_index': 'telegram-messages',
   '_id': '1v1XmpEBhq3eaWQ5uXob',
   '_score': 1.777031,
   '_source': {'msg_date': '2024-08-03T16:56:30',
    'msg_text': 'В том году ездили осенью. Можно договориться на месте, можно предварительно написать в ватсап. Они часов с 8-9 начинают работать, лучше к этому времени прие